In [ ]:
pip install -U google-generativeai #instala googleai para buscar las recetas
pip install openai #instala openai para generar la imagen

import google.generativeai as genai #importa gemini
import openai #importa openai
import requests #necesario
import os #necesario

In [ ]:
openai.api_key = "" #configura tu apikey de openia
genai.configure(api_key="") #configura tu apikey de gemini

In [ ]:
#modelo de google
generation_config = {
    "temperature": 0.5,
    "top_p": 0.95,
    "top_k": 64,
    "max_output_tokens": 8192,
    "response_mime_type": "text/plain",
}
#en el system_instruction se da el contexto, en este caso es un openbook porque le pido que busque en su base de datos la receta.
model = genai.GenerativeModel(
    model_name="gemini-1.5-flash",
    generation_config=generation_config,
    system_instruction="Eres un manager de fútbol un profesional como Pep Guardiola el hombre que más sabe de fútbol, y recomendará a los mejores jugadores para la posición del campo solicitada, dando datos como su valor, edad y habilidad con el balón. Además generas una imagen de el jugador con otro modelo por lo que si el usuario te dice no después de la primera recomendación es por que no requirió la imagen.",
)


In [ ]:
history_chat = []
# funcion para generar la imagen desde openai
def generate_image_with_openai(prompt):
    image_response = openai.Image.create(
        prompt=prompt,
        n=1,
        size="1024x1024"
    )

    # retorna la imagen en una url
    response = image_response['data'][0]['url']
    return response

#funcion para iniciar el chat en un bucle while
def chat():
    chat_session = model.start_chat(history=history_chat)
    
    print("Soy Pep Guardiola un entrenador de fútbol. Doy recomendaciones de jugadors que puedes necesitar para tu equipo, adelante pideme la posición del campo en la que te interesa fichar a un jugador.")
    
    while True:

        user_message = input("Tu: ")

        if user_message.lower() == "salir":  # Salir del chat
            break

        response = chat_session.send_message(user_message)
        print(response.text)  

        # Actualizar historial con la respuesta de Gemini
        history_chat.append({'role': 'user', 'parts': [user_message]})
        history_chat.append({'role': 'model', 'parts': [response.text]})

        # Después de la primera respuesta de gemini, se pregunta si se debe generar una imagen
        if len(history_chat) == 2:  # Verifica que es la primera interacción completa osea, usuario y modelo.
            #pregunta al usuario mediante un si o no si requiere la imagen o no.
            generate_image = input("¿Deseas generar una imagen con OpenAI basada en tu mensaje? (sí/no): ").strip().lower()
            
            if generate_image == "sí" or generate_image == "si":
                image_url = generate_image_with_openai(user_message)
                print(f"Image URL: {image_url}")
            elif generate_image == "no":
                print("Elija otra posicion del campo o otra opcion de jugador, si desea salir escriba 'salir' ")

            else:
                print("Respuesta no válida. Por favor, responde con 'sí' o 'no'.")

                # Agregar la URL de la imagen al historial
                history_chat.append({'role': 'model', 'parts': [f"Image URL: {image_url}"]})

chat()